In [10]:
import pandas as pd
import numpy as np
from dataloading import DataLoader
from datapreparation_py import DataPreparation
from eda_py import EDA
from eALS_adaptor import eALSAdaptor
from implicit.evaluation import train_test_split, ranking_metrics_at_k
from cv_py import CrossValidation
from visualize_py import Visualize

%cd C:\Users\781110104\OneDrive - Genpact\Documents\VSCode

C:\Users\781110104\OneDrive - Genpact\Documents\VSCode


In [73]:
dl = DataLoader()

In [74]:
#user_item_co = dl.import_data('AGCO', 'CO', 'df')
user_item_co_t = dl.import_data('TEREX', 'CO', 'df')

In [75]:
#user_item_filtered = dl.remove_low_interact_items(user_item_co, 1)
user_item_filtered_t = dl.remove_low_interact_items(user_item_co_t, 1)
#user_item_filtered_log = dl.log_scale_df(user_item_filtered, 0.01)
user_item_filtered_log_t = dl.log_scale_df(user_item_filtered_t, 0.01)
#user_item_csr = dl.to_csr(user_item_filtered_log)
user_item_csr_t = dl.to_csr(user_item_filtered_log_t)

In [5]:
# initial split
#train, test = train_test_split(user_item_csr, 0.8, 22)
train_t, test_t = train_test_split(user_item_csr_t, 0.8, 22)

In [8]:
cv = CrossValidation(5)
#validate, calibrate = cv.split_k_fold(22)
validate_t, calibrate_t = cv.split_k_fold(train_t, 22)

In [9]:
factors = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 110, 120, 130, 140, 150, 160, 170, 180, 190, 200]
iterations = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30]

In [246]:
space_iALS = {'factors' : factors, 'regularization' : [90], 'alpha' : [0.3], 'iterations' : iterations}
hyper_ials_add = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_iALS, model_class='iALS')
hyper_ials_add.to_csv('curves_ials_add.csv')

In [247]:
space_iALS_t = {'factors' : factors, 'regularization' : [90], 'alpha' : [1.3], 'iterations' : iterations}
hyper_ials_t_add = cv_t.hyperp_tuning(test=validate_t, train=calibrate_t, exclude=test_t, seed=22, param_space=space_iALS_t, model_class='iALS')
hyper_ials_t_add.to_csv('curves_ials_t_add.csv')

In [248]:
space_eALS_t = {'factors' : factors, 'regularization' : [50], 'alpha' : [0.1], 'w0' : [5000], 'iterations' : iterations}
hyper_eals_t_add = cv_t.hyperp_tuning(test=validate_t, train=calibrate_t, exclude=test_t, seed=22, param_space=space_eALS_t, model_class='eALS')
hyper_eals_t_add.to_csv('curves_eals_add.csv')

converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items to <class 'numpy.float32'>
converting type of user_items t

In [249]:
space_BPR = {'factors' : factors, 'regularization' : [0.03], 'learning_rate' : [0.005], 'iterations' : iterations}
hyper_bpr_add = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_BPR, model_class='BPR')
hyper_bpr_add.to_csv('curves_bpr_add.csv')

In [250]:
space_LMF = {'factors' : factors, 'regularization' : [40], 'learning_rate' : [1.2], 'iterations' : iterations, 'neg_prop': [50]}
hyper_lmf_add = cv.hyperp_tuning(test=validate, train=calibrate, exclude=test, seed=22, param_space=space_LMF, model_class='LMF')
hyper_lmf_add.to_csv('curves_lmf_add.csv')

Loading FM feature data:

In [76]:
dp = DataPreparation(user_item_filtered_log_t)

In [77]:
interactions_csr, weights_csr = dp.get_interaction_data()

In [14]:
item_features = ['group2', 'movement_code']
feature_input = dp.get_feature_data(OEM='TEREX', user_features=None, item_features=item_features)

In [78]:
train_t_w, test_t_w = train_test_split(weights_csr, 0.8, 22)
train_t_i, test_t_i = train_test_split(interactions_csr, 0.8, 22)

In [79]:
cv = CrossValidation(5)
#validate, calibrate = cv.split_k_fold(22)
validate_t_w, calibrate_t_w = cv.split_k_fold(train_t_w, 22)
validate_t_i, calibrate_t_i = cv.split_k_fold(train_t_i, 22)

In [80]:
itemf = feature_input

In [41]:
space_FM_warp = {'factors' : factors, 'regularization' : [0.005], 'learning_rate' : [0.1], 'iterations' : iterations, 
'loss' : ['warp'], 'max_sampled' : [50]}

uf_combi = [None]
if_combi = [itemf[0]]

uf_names = ['None']
if_names = ['itemf']

In [81]:
cal_w, val_w = train_test_split(train_t_w, 0.8, 22)

In [91]:
space_FM_warp = {'factors' : [40], 'regularization' : [0.005], 'learning_rate' : [0.10], 'iterations' : [10], 
'loss' : ['warp'], 'max_sampled' : [50]}
test_res_1 = cv.tune_FM(space=space_FM_warp, user_f=uf_combi, item_f=if_combi, uf_names=uf_names, if_names=if_names, train=cal_w, test=val_w, exclude= test_t_w)

In [92]:
test_res_1

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr,uf_name,if_name
0,40,0.005,0.1,10,warp,50,0.4225,0.290987,0.450773,0.511521,0.1793,None,itemf


In [67]:
space_FM_warp = {'factors' : [64], 'regularization' : [0.005], 'learning_rate' : [0.1], 'iterations' : [15], 
'loss' : ['warp'], 'max_sampled' : [50]}
test_res = cv.hyperp_tuning(validate_t_w, calibrate_t_w, test_t_w, 22, space_FM_warp, 'FM', 'full', None, itemf[0])

In [68]:
test_res

,factors,regularization,learning_rate,iterations,loss,max_sampled,precision,map,ndcg,auc,mpr
0,64,0.005,0.1,15,warp,50,0.4195,0.283052,0.43544,0.511861,0.188064


In [90]:
test_t_w.data

array([4.61512052, 5.99396143, 6.39859493, ..., 7.93773178, 4.61512052,
       5.99396143])

In [238]:
# saving locally
hyper_ials.to_csv('curves_ials.csv')
hyper_ials_t.to_csv('curves_ials_t.csv')
hyper_eals_t.to_csv('curves_eals.csv')
hyper_bpr.to_csv('curves_bpr.csv')
hyper_lmf.to_csv('curves_lmf.csv')